<h1><center>Plan</center></h1>

| <h2>Classification</h2> | <h2>Regression</h2> | <h2>Clustering</h2> |
| :- | :- | :- |
| Neural network (MLP) DONE | Decision tree (LightGBM) DONE | PCA and k-means DONE |
| Decision tree (LightGBM) DONE | Neural network (MLP) DONE |  |
| kNN DONE| kNN DONE |  |



# Load the training data

In [1]:
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler

In [2]:
def load_data(name):
    with h5py.File(f'{name}.h5', 'r') as f:
        return pd.DataFrame(f[name][:])

train = load_data('train')

#only use 6 features (curse of dimensionality)
best_features = ['p_sigmad0',
 'p_ethad',
 'p_Rhad',
 'p_Rphi',
 'p_Reta',
 'p_deltaEta1']


train_variables = train[best_features]
train_energy = train['p_truth_E']
train_class = train['Truth']

#scale data
transformer = RobustScaler().fit(train_variables)
train_variables = pd.DataFrame(transformer.transform(train_variables), columns=train_variables.columns
)

# Classification

### kNN (SciKit-learn)

In [3]:
#find optimal k
from sklearn.neighbors import KNeighborsClassifier as kNN
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

ks = [11, 13, 15, 17]
k_acc = []
for k in ks: 
    fold_acc = []
    
    cv = KFold(n_splits=5)
    for train_index, validation_index in cv.split(train_variables.to_numpy()):
        
        #define training and validation sets for each fold
        x_trainCV, x_valCV, y_trainCV, y_valCV = train_variables.to_numpy()[train_index], train_variables.to_numpy()[validation_index], train_class.to_numpy()[train_index], train_class.to_numpy()[validation_index]

        #train kNN model on training set
        neigh = kNN(n_neighbors=k)
        neigh.fit(x_trainCV, y_trainCV)
        
        #append accuracy for each fold
        y_pred = neigh.predict(x_valCV)
        acc = accuracy_score(y_pred, y_valCV)
        fold_acc.append(acc)
   
    #append mean accuracy for each leaf number    
    k_acc.append(np.mean(fold_acc))

plt.scatter(ks, k_acc)
plt.show()

# #15 seems to be the best k

In [4]:
#use optimal k 
x_train, x_val, y_train, y_val = train_test_split(train_variables, train_class, test_size=0.3, random_state=42)

neigh = kNN(n_neighbors=15)
neigh.fit(x_train, y_train)
y_pred = neigh.predict(x_val)
acc = accuracy_score(y_pred, y_val)
y_pred_prob = neigh.predict_proba(x_val)
print('kNN accuracy score:'+str(acc))

kNN accuracy score:0.9223589743589744


# Test data

In [5]:
#load test set
test = load_data('test')
test_variables = test[best_features]

#scale test data
transformer = RobustScaler().fit(test_variables)
test_variables = pd.DataFrame(transformer.transform(test_variables), columns=test_variables.columns
)

In [7]:
#use model on test data
y_pred_prob = pd.DataFrame(neigh.predict_proba(test_variables)[:,1])

#export to csv
# y_pred_prob.to_csv('predicted_classes_kNN.csv', header=False)